In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

import os

DATA_DIR = os.path.join(
    path,
    "asl_alphabet_train",
    "asl_alphabet_train"
)

Path to dataset files: C:\Users\DOAN CAO THANH TUYEN\.cache\kagglehub\datasets\grassknoted\asl-alphabet\versions\1


# Import du lieu

In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Chuan hoa du lieu

In [22]:
IMG_SIZE = 64
BATCH_SIZE = 32

In [23]:
# cai thien datagen
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

In [24]:
train_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # chi dinh tap train
)

val_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # chi dinh tap validation
)

Found 69623 images belonging to 2 classes.
Found 17405 images belonging to 2 classes.


# Khai bao mo hinh - fit

In [25]:
# Add BatchNormalization and stronger regularization
from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # increase dropout
    Dense(29)  # no activation here, use from_logits=True
])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,468,829 (9.42 MB)

 Trainable params: 2,468,381 (9.42 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# huan luyen
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

# Save model

In [ ]:
model.save('asl_alphabet_model.h5') # .h5: dung de load lai va du doan cho sau nay
print('Saved model')

# Danh gia mo hinh

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {loss:.4f}')

# Du doan

In [ ]:
# load mo hinh
loaded_model = keras.models.load_model('asl_alphabet_model.h5')
print('Loaded model from disk')

In [ ]:
# chuan bi label (chu cai)
class_names = list(train_data.class_indices.keys())
print(class_names)

In [ ]:
# load 1 hiunh de xu ly
img_path = path + '/test.jpg' # duong dan den hinh can du doan
# img_path = input("Enter the path of the image to predict")
img = imgage.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = img.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0)  # them kich thuoc 

In [ ]:
# du doan
predictions = loaded_model.predict(img_array)
predicted_index = np.argmax(predictions)
predicted_label = class_names[predicted_index] # map index -> label

# print % cac lop du doan
print(predictions)
print(f'Predicted label: {predicted_label}')